In [7]:
import copy
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
import numpy as np
import math
import glob, random, time, sys
from string import printable
from datetime import datetime
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge

from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.metrics import make_scorer


from sklearn.linear_model import Lasso, RidgeClassifier, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.metrics import mean_squared_error, log_loss, accuracy_score, mean_absolute_error
from sklearn.datasets import load_diabetes, load_iris
from sklearn.neighbors import KNeighborsRegressor

from HYBparsimony import Population, HYBparsimony, order
from HYBparsimony.hybparsimony import default_cv_score_classification
from HYBparsimony.util import knn_complexity

import seaborn as sns
import pickle

# from GAparsimony import GAparsimony, Population, getFitness
# from GAparsimony.util import linearModels_complexity, svm_complexity, mlp_complexity, randomForest_complexity
from matplotlib import pyplot as plt
import sklearn

# from PSOparsimony_Nested import PSOparsimony
from bayes_opt import BayesianOptimization
# from HYBparsimony import HYBparsimony
import warnings
from sklearn.exceptions import ConvergenceWarning
from IPython.display import clear_output

# from Hyb_nestedCV import HYB_NestedCV
warnings.simplefilter("ignore", category=ConvergenceWarning)

# comment in Beronia (not installed)
from autogluon.tabular import TabularDataset, TabularPredictor
print(sklearn.__version__, pd.__version__, np.__version__)



1.1.3 1.5.2 1.23.5


In [8]:
# REGRESIÓN
# Cargo un dataset de regresión
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

###############################################################
#                EJEMPLO OTRO SCORING Y OTRO CV               #
###############################################################
KNN_Model = {"estimator": KNeighborsRegressor,
                 "complexity": knn_complexity,
                 "n_neighbors": {"range": (1, 10), "type": Population.INTEGER}
                 }
HYBparsimony_model = HYBparsimony(scoring="neg_mean_squared_error", 
                                  algorithm ='KernelRidge',
                                  cv = RepeatedKFold(n_splits=10, n_repeats=5), 
                                  n_jobs = 1,
                                  rerank_error = 0.1, 
                                  features = diabetes.feature_names)
HYBparsimony_model.fit(X_train, y_train, time_limit=0.2)
preds = HYBparsimony_model.predict(X_test)
print("RMSE test", mean_squared_error(y_test, preds, squared=False))
    

Running iteration 0
Current best score: -3004.3006761612755
Running iteration 1
Current best score: -2990.7795720943313
Time limit reached. Stopped.
Selected features: ['age' 'sex' 'bmi' 'bp' 's1' 's2' 's4' 's5' 's6']
RMSE test 53.60827964891579


## Default

In [ ]:
# Argumentos
# ----------
def in_nb():
    import __main__ as main
    return not hasattr(main, '__file__')


import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--method", type=str, default="PSO")
parser.add_argument("--database", type=str, default="boston_norm.csv")
parser.add_argument("--algorithm", type=str, default="")  # kridge, mlp, rf, svr
parser.add_argument("--GPU", type=str, default="False")
parser.add_argument("--nump", type=int, default=40)
parser.add_argument("--maxiter", type=int, default=200)
parser.add_argument("--early", type=int, default=35)
parser.add_argument("--pcrossover_elitists", type=float, default=None)
parser.add_argument("--pcrossover", type=float, default=None)
parser.add_argument("--Lambda", type=float, default=1.0)
parser.add_argument("--c1", type=float, default=1.193)
parser.add_argument("--c2", type=float, default=1.193)
parser.add_argument("--IWmax", type=float, default=0.9)
parser.add_argument("--IWmin", type=float, default=0.4)
parser.add_argument("--K", type=int, default=3)
parser.add_argument("--globthr", type=float, default=1.0)
parser.add_argument("--CVrep", type=int, default=5)
parser.add_argument("--CVspl", type=int, default=5)
parser.add_argument("--nruns", type=int, default=5)
parser.add_argument("--numseeds", type=int, default=5)
parser.add_argument("--timelimitmin", type=float, default=1)
parser.add_argument("--rerank", type=float, default=0.001)
parser.add_argument("--dirout", type=str, default="respso2")
parser.add_argument("--not_muted", type=int, default=3)
parser.add_argument("--n_jobs", type=int, default=1)
parser.add_argument("--n_jobs_autogluon", type=int, default=64)
parser.add_argument("--temp_autogluon", type=str, default="temp_autogluon")
parser.add_argument("--train_size", type=int, default=2000)

PRUEBA = 'HYB'
if PRUEBA == 'HYB':
    # PSO
    cmdline = "--method HYB --rerank 0.001 --nump 15 --numseeds 5 --pcrossover 0.10"
    cmdline += " --c1 1.193 --c2 1.193 --maxiter 200 --n_jobs 1 --dirout hyb_1001_12abr23_e27 --algorithm kridge"
    cmdline = cmdline.split(' ')
    
if PRUEBA == 'PSO':
    # PSO
    cmdline = "--method PSO --rerank 0.001 --nump 15 --numseeds 5 --pcrossover 0.10"
    cmdline += " --c1 1.193 --c2 1.193 --maxiter 200 --n_jobs 1 --dirout pso_16_22mar23 --algorithm kridge"
    cmdline = cmdline.split(' ')
    
if PRUEBA == 'BAYESOPT':
    # Bayesian Optimization
    cmdline = "--database BD_soldadura_20feb23.csv --method BAYESOPT --algorithm kridge"
    cmdline += " --maxiter 200 --nruns 10 --dirout bayes_kridge_10runs --n_jobs 1"
    cmdline = cmdline.split(' ')

if PRUEBA == 'CONSOLE':
    cmdline = sys.argv[1:]  # Read from console

arg_in = parser.parse_args(cmdline)
str_todos = '_'.join(str(arg_in).split(' '))
str_todos = str_todos.replace(',', '')
str_todos = str_todos.replace('Namespace(', '')
str_todos = str_todos.replace(')', '')
str_todos = str_todos.replace('=', '_')
str_todos = str_todos.replace('.', '_')
str_todos = str_todos.replace("'", "")
str_todos = str_todos.replace('\\r', '')
print(str_todos)

# Decreasing curve of worst
if arg_in.pcrossover != None and arg_in.pcrossover != 0.0:
    perc_malos = 0.80 * np.exp(-arg_in.pcrossover * np.arange(arg_in.maxiter))
    perc_malos[perc_malos < 0.10] = 0.10
else:
    perc_malos = None
    
if arg_in.GPU == 'True':
    arg_in.GPU = True
    IS_GPU = 'YES'
else:
    arg_in.GPU = False
    IS_GPU = 'NO'
print('Using GPU=', IS_GPU)

In [ ]:
now = datetime.now()
dt_string = now.strftime("%Y_%m_%d_%H_%M_%S_")
NAME_FILE = dt_string + '_' + arg_in.method + '__' + arg_in.algorithm + '__' + '__' + str(arg_in.nump)  # .join(arg_in[:2]) #+ str_todos
NAME_FILE = ''.join(char for char in NAME_FILE if char in printable)
print(repr(NAME_FILE))

NUM_PARTICULAS = arg_in.nump
DIR_SALIDA = str(arg_in.dirout).replace('\\r', '').replace('\r', '')
DIR_SALIDA = ''.join(char for char in DIR_SALIDA if char in printable)
print(repr(DIR_SALIDA))

if not os.path.exists(DIR_SALIDA):
    os.mkdir(DIR_SALIDA)

## Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

def rmse_func(y_true, y_pred):
    return -np.sqrt(mean_squared_error(y_true, y_pred))

def decision_trees_complexity(model, nFeatures, **kwargs):
    num_leaves = model.get_n_leaves()
    int_comp = np.min((1E09 - 1, num_leaves))  # More leaves more complex
    return nFeatures * 1E09 + int_comp

def randomForest_complexity(model, nFeatures, **kwargs):
    #     num_leaves = [tree.get_n_leaves() for tree in model.estimators_]
    #     int_comp = np.min((1E09-1,np.mean(num_leaves))) # More leaves more complex
    max_depth = model.get_params()['max_depth']
    int_comp = max_depth  # Internal Complexity
    return nFeatures * 1E09 + int_comp

def kernel_ridge_complexity(model, nFeatures, **kwargs):
    weights_l2 = np.sum(model.dual_coef_ ** 2)
    int_comp = np.min((1E09 - 1, weights_l2))  # More leaves more complex
    return nFeatures * 1E09 + int_comp

def mlp_complexity_hidden(model, nFeatures, **kwargs):
    weights = [np.concatenate(model.intercepts_)]
    for wm in model.coefs_:
        weights.append(wm.flatten())
    weights = np.concatenate(weights) 
    int_comp = np.min((1E09-1,np.sum(weights**2)))
#     int_comp = model.hidden_layer_sizes
    return nFeatures*1E09 + int_comp

In [ ]:
def fitness_model(cromosoma, X_train, y_train, X_val, y_val, iteration, particle, **kwargs):
        global historical_todos, innerfold, outerfold, target_name
        global algorithm, metric, complexity
        # Todas las feats (hay que poner selec features en otros problemas)
        num_feats = X_train.shape[1]
        parametros = cromosoma.params.copy()
        selection = cromosoma.columns
        # train the model
        if arg_in.algorithm == 'mlp':
            parametros['alpha'] = 10.0**parametros['alpha']
            parametros['hidden_layer_sizes'] = int(parametros['hidden_layer_sizes'])
        
        if arg_in.algorithm == 'kridge':
            parametros['alpha'] = 10.0**parametros['alpha']
            parametros['gamma'] = 10.0**parametros['gamma']

        modelo = algorithm(**parametros).fit(X_train.values[:,selection], y_train)
        fitness_train = metric(y_train.values.flatten(), modelo.predict(X_train.values[:,selection]))
        fitness_val = metric(y_val.values.flatten(), modelo.predict(X_val.values[:,selection]))
        fitness_test = metric(y_test.values.flatten(), modelo.predict(X_test.values[:,selection]))
#         historical_todos.append(dict(target_name=target_name, outerfold=outerfold, innerfold=innerfold,
#                                      iteration=iteration, num_indiv=particle, 
#                                      fitness_train= -fitness_train, fitness_val= -fitness_val, fitness_test= -fitness_test,
#                                      num_feats=num_feats, parametros=parametros,
#                                      feats = cromosoma.columns#, feats_prob = feats_prob.tolist()
#                                      ))
        return np.array([fitness_train, fitness_val, complexity(modelo, np.sum(cromosoma.columns))]), modelo

In [ ]:
# def getFitness_BayesOpt(algorithm, metric, complexity, cv=RepeatedKFold(n_splits=10, n_repeats=5, random_state=42),
#                           n_jobs=-1):
#          def fitness(X_train, X_val, y_train, y_val, cromosoma, **kwargs):
#             global mejor_train, mejor_val, mejor_tst, parametros_mejor, num_indiv, columnas_mejor, target_name, historical_todos, run, numseeds, mejor_nfs
#             global input_names, target_name, mejor_solucion_es, X_test, y_test

#             X_train = X_train.values
#             y_train = y_train.values
#             X_val = X_val.values
#             y_val = y_val.values
#             data_train_model = X_train[:, cromosoma.columns]
#             data_val_model = X_val[:, cromosoma.columns]

#             num_feats = np.sum(cromosoma.columns)
#             # train the model
#             parametros = cromosoma.params.copy()
#             print('Var=', target_name, 'Seed=', numseeds, 'Run=', run, 'Iter=', num_indiv // NUM_PARTICULAS, 'Nindiv=',
#                   num_indiv % NUM_PARTICULAS,
#                   'NFs=', num_feats, parametros)
#             print(cromosoma.columns)
#             print(arg_in.algorithm, ' : mejor_val=', mejor_val, 'mejor_tst=', mejor_tst, 'numfs=', mejor_nfs, 'params=',
#                   parametros_mejor)

#             #         try:
#             aux = algorithm(**parametros)

#             fitness_train = cross_val_score(aux, data_train_model, y_train, scoring=make_scorer(metric), cv=cv, n_jobs=n_jobs).mean()

#             modelo = algorithm(**parametros).fit(data_train_model, y_train)
#             fitness_val = metric(modelo.predict(data_val_model), y_val)
#             # Esto es para ver cómo se comporta el individuo contra el otro 50% (la BD de test)
#             fitness_test = metric(modelo.predict(X_test.values[:,cromosoma.columns]), y_test)

#             print("FITNESS_VAL =", fitness_val)
#             if -fitness_val < mejor_val:
#                 mejor_train = -fitness_train
#                 mejor_val = -fitness_val
#                 mejor_tst = -fitness_test
#                 mejor_nfs = num_feats
#                 parametros_mejor = parametros
#                 columnas_mejor = cromosoma.columns
#             print('fitness_val=', -fitness_val)
#             mejor_solucion_es = arg_in.algorithm + ' : mejor_val=' + str(mejor_val) + ' mejor_tst=' + str(
#                 mejor_tst) + 'numfs=' + str(mejor_nfs) + ' params=' + str(parametros_mejor)
#             # Aquí divido entre 10 porque hay 10 puntos iniciales al hacer BayesOpt.
#             historical_todos.append(dict(target_name=target_name, iteracion=num_indiv // 10,
#                                          num_indiv=num_indiv % 10,
#                                          mejor_train = mejor_train, mejor_val = mejor_val, mejor_test=mejor_tst,
#                                          fitness_train=-fitness_train, fitness_val=-fitness_val, fitness_test=-fitness_test,
#                                          num_feats=num_feats, parametros=parametros, numseeds=numseeds, run=run,
#                                          mejor_solucion_es=mejor_solucion_es, feats = cromosoma.columns))
#             num_indiv += 1
#             #fitnessval es negativo. Y nos interesa que lo sea, porque el Bayesian Opt va a optimizar maximizando..
#             return np.array(
#                 [fitness_train, fitness_val, complexity(modelo, np.sum(cromosoma.columns))]), modelo, mejor_solucion_es

#         #         except Exception as e:
#         #             print(e)
#         #             return np.array([np.NINF, np.NINF, np.Inf]), None
#         return fitness

In [ ]:
def fitness_nested(algorithm_indiv, metric_indiv, complexity_indiv):
    global algorithm, metric, complexity
    if algorithm_indiv is None:
        raise Exception("An algorithm function must be provided!!!")
    if metric_indiv is None or not callable(metric_indiv):
        raise Exception("A metric function must be provided!!!")
    if complexity_indiv is None or not callable(complexity_indiv):
        raise Exception("A complexity function must be provided!!!")
    algorithm = algorithm_indiv
    metric = metric_indiv
    complexity = complexity_indiv
    return fitness_model

def fitness_nested_for_parallel(algorithm_indiv, metric_indiv, complexity_indiv, cromosoma, X_train, y_train, X_val, y_val, iteration, particle):
    global algorithm, metric, complexity
    if algorithm_indiv is None:
        raise Exception("An algorithm function must be provided!!!")
    if metric_indiv is None or not callable(metric_indiv):
        raise Exception("A metric function must be provided!!!")
    if complexity_indiv is None or not callable(complexity_indiv):
        raise Exception("A complexity function must be provided!!!")
    algorithm = algorithm_indiv
    metric = metric_indiv
    complexity = complexity_indiv
    return fitness_model(cromosoma, X_train, y_train, X_val, y_val, iteration, particle, visualize=False)    

## Model Parameters

In [ ]:
from functools import partial

if arg_in.algorithm == 'kridge':
    
    fitness_fun = fitness_nested(KernelRidge, rmse_func, kernel_ridge_complexity)
    params = {"alpha": {"range": (-7, 0), "type": Population.FLOAT},
              "gamma": {"range": (-7, 0), "type": Population.FLOAT},
              "kernel": {"value": "rbf", "type": Population.CONSTANT}}

if arg_in.algorithm == 'mlp':
    if arg_in.n_jobs<=1:
        fitness_fun = fitness_nested(MLPRegressor, rmse_func, mlp_complexity_hidden)
    else:
        fitness_fun = partial(fitness_nested_for_parallel, 
                          MLPRegressor, 
                          rmse_func, 
                          mlp_complexity_hidden)
    params = {"hidden_layer_sizes": {"range": (1, 25), "type": Population.INTEGER},  # MLPRegressor
              "alpha": {"range": (-7, 0), "type": Population.FLOAT},
              "solver": {"value": "lbfgs", "type": Population.CONSTANT},
              "activation": {"value": "logistic", "type": Population.CONSTANT},
              "n_iter_no_change": {"value": 20, "type": Population.CONSTANT},
              "tol": {"value": 1e-5, "type": Population.CONSTANT},
              "random_state": {"value": 1234, "type": Population.CONSTANT},
              "max_iter": {"value": 5000, "type": Population.CONSTANT}}

## Search Methods

In [ ]:
def search_model(output_var, seed_ini=1234, X_train=None, X_val=None, y_train=None, y_val=None,
                 X_train_val = None, y_train_val = None, features = None):

    global X_train_orig, X_test_orig, input_names, target_name, X, y
    global fitness_fun, params, arg_in
    if arg_in.method == 'PSO' or arg_in.method == 'HYB':
        PSOparsimony_model = PSOparsimony(fitness=fitness_fun,
                                          params=params,
                                          features=input_names,
                                          keep_history=True,
                                          rerank_error=arg_in.rerank,
                                          npart=arg_in.nump,
                                          maxiter=arg_in.maxiter,
                                          early_stop=arg_in.early,
                                          feat_thres=0.90,  # Perc selected features in first generation
                                          seed_ini=seed_ini,
                                          Lambda=arg_in.Lambda,
                                          c1=arg_in.c1,
                                          c2=arg_in.c2,
                                          IW_max=arg_in.IWmax,
                                          IW_min=arg_in.IWmin,
                                          K=arg_in.K,
                                          best_global_thres=arg_in.globthr,
                                          # pcrossover_elitists=arg_in.pcrossover_elitists,
                                          pcrossover=perc_malos,
                                          n_jobs=arg_in.n_jobs,
                                          verbose=0)
        PSOparsimony_model.fit(X_train = X_train[features], y_train = y_train,
                               X_val = X_val[features], y_val = y_val)
        return PSOparsimony_model
    
    if arg_in.method == 'BAYESOPT':
        global NUM_PARTICULAS
        NUM_PARTICULAS = arg_in.maxiter

        pbounds = dict()
        for parm in params:
            if params[parm]['type'] <= 1:
                pbounds[parm] = params[parm]['range']

        if arg_in.algorithm == 'kridge':
            def optimize_fun(alpha, gamma):
                chromosome_all_ones = pd.Series(
                    dict(columns=(np.ones(len(input_names)) == 1), params=dict(alpha=alpha, gamma=gamma, kernel="rbf")))
                res = fitness_fun(chromosome_all_ones, X_train, y_train, X_val, y_val, 0, 0)
                return res[0][1] # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.
        if arg_in.algorithm == 'mlp':
            def optimize_fun(hidden_layer_sizes, alpha):
                params_opt = dict(hidden_layer_sizes=int(hidden_layer_sizes),
                                  alpha=alpha,
                                  solver='lbfgs',
                                  activation='logistic',
                                  n_iter_no_change=20,
                                  tol=1e-5,
                                  random_state=1234,
                                  max_iter=5000)
                chromosome_all_ones = pd.Series(dict(columns=(np.ones(len(input_names)) == 1), params=params_opt))
                res = fitness_fun(chromosome_all_ones, X_train, y_train, X_val, y_val, 0, 0)
                return res[0][1] # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.

        # Optimize with BayesianOptimization
        tic = time.time()
        optimizer = BayesianOptimization(f=optimize_fun,
                                         pbounds=pbounds,
                                         random_state=seed_ini,
                                         allow_duplicate_points=True) # Añado allow_duplicate_points=True para evitar errores que salían.
        optimizer.maximize(init_points=10, n_iter=arg_in.maxiter - 10)
        # Keep elapsed time in minutes
        tac = time.time()
        elapsed_time = (tac - tic) / 60.0
        print('Elapsed_time:', elapsed_time)
        print('Best:', optimizer.max)
        
        # Best model
        # ----------
        # KRIDGE
        if arg_in.algorithm == 'kridge':
            alpha = optimizer.max['params']['alpha']
            gamma = optimizer.max['params']['gamma']
            params_opt = dict(alpha=alpha, gamma=gamma, kernel="rbf")
            params_indiv = [optimizer.max['params']['alpha'], optimizer.max['params']['gamma']]

        # MLP
        if arg_in.algorithm == 'mlp':
            hidden_layer_sizes = optimizer.max['params']['hidden_layer_sizes']
            alpha = optimizer.max['params']['alpha']
            params_opt = dict(hidden_layer_sizes=int(hidden_layer_sizes),
                              alpha=alpha,
                              solver='lbfgs',
                              activation='logistic',
                              n_iter_no_change=20,
                              tol=1e-5,
                              random_state=1234,
                              max_iter=5000)
            params_indiv = [optimizer.max['params']['hidden_layer_sizes'], optimizer.max['params']['alpha']]

        # Return results with the best model
        chromosome_all_ones = pd.Series(dict(columns=(np.ones(len(input_names)) == 1), params=params_opt))
        res = fitness_fun(chromosome_all_ones, X_train, y_train, X_val, y_val, 0, 0)
        return pd.Series(dict(best_model=res[1],
                              bestsolution=np.concatenate([np.array(res[0]), params_indiv, np.ones(len(input_names))]),
                              minutes_total=elapsed_time))

## Search 

### Search with Nested CV

In [ ]:
tiempos = [['slice_norm_reduc.csv', 3, 2000, 378, 0.50], 
#            ['blog_norm.csv', 3, 2000, 276, 0.50], 
           ['crime_norm.csv', 3, 2000, 127, 0.50], 
#            ['tecator_norm.csv', 3, 2000, 124, 0.50],
           ['ailerons_norm.csv', 3, 2000, 40, 0.50], 
#            ['puma_norm.csv', 3, 2000, 32, 0.50],
           ['bank_norm.csv', 3, 2000, 32, 0.50], 
#            ['pol_norm.csv', 3, 2000, 26, 0.50],
           
#            ['cpu_act_norm.csv', 3, 2000, 21, 0.50],
#            ['elevators_norm.csv', 3, 2000, 18, 0.50],
#            ['meta_norm.csv', 3, 2000, 17, 0.50],
#            ['bodyfat_norm.csv', 3, 2000, 14, 0.50], 
#            ['boston_norm.csv', 3, 2000, 13, 0.50],
#            ['housing_norm.csv', 3, 2000, 13, 0.50],
#            ['concrete_norm.csv', 3, 2000, 8, 0.50],
#            ['no2_norm.csv', 3, 2000, 7, 0.50],
#            ['pm10_norm.csv', 3, 2000, 7, 0.50],
#            ['strike_norm.csv', 3, 2000, 6, 0.50]
          ]
    

# # From Lee_Synthetic_All_10runs_27mar.ipynb
# tiempos = [['ailerons_norm.csv', 3, 2000, 40, 1.0],
#            ['crime_norm.csv', 3, 2000, 127, 0.4], 
#            ['blog_norm.csv', 3, 2000, 276, 0.6],
#            ['slice_norm_reduc.csv', 3, 2000, 378, 0.60]]
    

run = 0
for names_tiempo in tiempos:
    name_db = names_tiempo[0].split('_')[0]
    list_csv = os.listdir('.')
    list_csv = [i for i in list_csv if f'{name_db}' in i]
    file_db = list_csv[0]
    time_limit = int(names_tiempo[1] * 60)  # Time limit in secs
    
    if arg_in.method == 'PSO':
        perc_malos = None
    else:
        # Calcula la curva de perc_malos
        perc_malos = 0.80 * np.exp(-names_tiempo[4] * np.arange(arg_in.maxiter))
        perc_malos[perc_malos < 0.10] = 0.10
        print(perc_malos)
    
#     train_val_size = names_tiempo[2]

    df_VAL = pd.read_csv(file_db)
    print(df_VAL.shape)
    train_val_size = df_VAL.shape[0]//2
    if train_val_size>2000:
        train_val_size = 2000
    names_tiempo[2] = train_val_size
    print(name_db, time_limit, train_val_size)
    
    res_modelos = []
    GA_models = []
    historical_todos = []
    tiempo_inicial = time.time()
    outer_res = []
    
    NAME_FILE = dt_string + '_' + arg_in.method + '__' + arg_in.algorithm + '__' + file_db + '__' + str(arg_in.nump)  # .join(arg_in[:2]) #+ str_todos
    NAME_FILE = ''.join(char for char in NAME_FILE if char in printable)
    print(repr(NAME_FILE))
    
    for num_seed in range(arg_in.numseeds):
        print('Processando Base de Datos con num_seed=', num_seed)
        df_VAL = pd.read_csv(file_db)
        VAL = df_VAL.copy()
        scaler_pruebas = StandardScaler()
        VAL_norm = pd.DataFrame(scaler_pruebas.fit_transform(VAL), columns=VAL.columns)

        TST_RMSE_ITER = []
        NUM_FEATS = []
        
        # Con selección
        input_names = VAL_norm.columns[:-1]
        target_name = VAL_norm.columns[-1]

        X = VAL_norm[input_names]
        y = VAL_norm[target_name]
        seed_everything(num_seed)

         # IMPORTANTE: En KFOLD_HYB_AUTOGLUON cojo 2000 filas de train_val y el resto de test...salvo que se especifique
        # el parámetro train_size (que será el número de filas a pillar de la BD).
        X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, train_size = names_tiempo[2] / len(X), 
                                                                    random_state=12345)
        size_train_val = len(X_train_val)
        size_test = len(X_test)

        model_name = 'KFOLD_HYB_AUTOGLUON'
        selec_input_folds=[]
        VAL_RMSE_ACC=[]
        # En este caso, hacemos los X folds (fold es el numseeds)
        kf = KFold(n_splits=arg_in.numseeds, shuffle=False)
        for innerfold, (train_index, val_index) in enumerate(kf.split(X_train_val)):
            print(f"Executing num_seed:{num_seed} innerfold:{innerfold}:")
            X_train = X_train_val.iloc[train_index]
            y_train = y_train_val.iloc[train_index]
            X_val = X_train_val.iloc[val_index]
            y_val = y_train_val.iloc[val_index]

            res = []
            mejor_train = np.inf
            mejor_val = np.inf
            mejor_tst = np.inf
            mejor_nfs = np.inf
            parametros_mejor = []
            mejor_solucion_es = 'EMPTY'
            columnas_mejor = []
            num_indiv = 0

            # Busco el mejor modelo para innerfold
            res_search_model = search_model(target_name, seed_ini=1234 * (innerfold + 1) * (num_seed + 1) * (run+1),
                                            X_train=X_train, y_train=y_train, 
                                            X_val=X_val, y_val=y_val,
                                            features=input_names)

            # Extrae Resultados del mejor modelo de cada innerfold
            time_limit = int(arg_in.timelimitmin * 60)  # Time limit in secs
            selec_input = input_names[res_search_model.bestsolution[-len(input_names):].astype(float) >= 0.50]
            print("SELECT INPUT", selec_input)
            selec_input_folds.append(selec_input)

            # ERRORES DE ENTRENAMIENTO, VALIDACIÓN Y TESTEO DE CADA FOLD:            
            # Extrae Resultados mejor Modelo
            model_name = str(res_search_model.best_model).split('(')[0] + '_' + target_name
            model = res_search_model.best_model

            # Entrenamos con TRAIN y medimos el error con VAL
            model.fit(X_train[selec_input].values, y_train.values)
            y_val_pred = model.predict(X_val[selec_input].values)
            VAL_RMSE_FOLD = mean_squared_error(y_val.values, y_val_pred, squared=False)
            VAL_MAE = mean_absolute_error(y_val.values, y_val_pred)
            VAL_MAPE = mean_absolute_percentage_error(y_val.values, y_val_pred) * 100
            y_test_pred = model.predict(X_test[selec_input].values)
            TST_RMSE_FOLD = mean_squared_error(y_test.values, y_test_pred, squared=False)

            # Entrenamos con TRAIN+VAL y medimos el error con TST
            model.fit(X_train_val[selec_input].values, y_train_val.values)
            y_test_pred = model.predict(X_test[selec_input].values)
            TST_RMSE = mean_squared_error(y_test.values, y_test_pred, squared=False)
            TST_MAE = mean_absolute_error(y_test.values, y_test_pred)
            TST_MAPE = mean_absolute_percentage_error(y_test.values, y_test_pred) * 100

            resultados_iter = dict(method=arg_in.method,
                                   database=arg_in.database,
                                   namevar=target_name,
                                   size_train_val=size_train_val,
                                   size_test=size_test,
                                   num_input_feats=X.shape[1],
                                   num_seed=num_seed,
                                   innerfold=innerfold,
                                   seed=num_seed, run=run, model_name=model_name,
                                   rerank_error=arg_in.rerank,

                                   VAL_RMSE_FOLD=VAL_RMSE_FOLD,
                                   VAL_MAE=VAL_MAE,
                                   VAL_MAPE=VAL_MAPE,
                                   TST_RMSE_FOLD=TST_RMSE_FOLD, #rmse with train
                                   TST_RMSE=TST_RMSE, #rmse with train+val
                                   TST_MAE=TST_MAE,
                                   TST_MAPE=TST_MAPE,

                                   COMPLEXITY=res_search_model.bestsolution[2],
                                   time=res_search_model.minutes_total,
                                   last_iter=res_search_model.iteration,
                                   mejor_val=mejor_val,
                                   numindiv=arg_in.nump,
                                   maxiter=arg_in.maxiter,
                                   early=arg_in.early,
                                   Lambda=arg_in.Lambda,
                                   c1=arg_in.c1,
                                   c2=arg_in.c2,
                                   IWmax=arg_in.IWmax,
                                   IWmin=arg_in.IWmin,
                                   K=arg_in.K,
                                   globthr=arg_in.globthr,
                                   CVrep=arg_in.CVrep,
                                   CVspl=arg_in.CVspl,
                                   nruns=arg_in.nruns,
                                   pcrossover_elitists=arg_in.pcrossover_elitists,
                                   pcrossover=arg_in.pcrossover,
                                   best_solution=res_search_model.bestsolution
                                   )

            # Escribimos en cada iteración en el CSV los resultados
            res_modelos.append(resultados_iter)
            res_final = pd.DataFrame(res_modelos)
            res_final.to_csv(DIR_SALIDA + '/' + NAME_FILE + '_results.csv', index=False)

        # FINAL MODEL FOR EACH OUTER WITH MEAN PARAMS
        if arg_in.algorithm == 'mlp':
            only_outer = res_final.query('num_seed==@num_seed').reset_index(drop=True)
            only_outer['complex_best'] =  only_outer['best_solution'].apply(lambda x: x[2]).values
            only_outer['hidden'] =  only_outer['best_solution'].apply(lambda x: x[3]).values
            only_outer['alpha'] =  only_outer['best_solution'].apply(lambda x: 10**x[4]).values
            mean_outer = only_outer[['VAL_RMSE_FOLD', 'VAL_MAE', 'VAL_MAPE', 'TST_RMSE_FOLD',
                                     'TST_RMSE', 'TST_MAE', 'TST_MAPE',
                                    'complex_best', 'hidden', 'alpha']].aggregate(['mean','std'])
            fil0 = mean_outer.iloc[0]
            fil0.index = ['mean_'+col for col in mean_outer.columns]
            fil1 = mean_outer.iloc[1]
            fil1.index = ['std_'+col for col in mean_outer.columns]
            mean_outer = pd.concat(
                [only_outer[['method', 'database', 'namevar', 'size_train_val', 'size_test',
                       'num_input_feats', 'num_seed', 'innerfold', 'run', 'model_name',
                       'rerank_error', 'time', 'last_iter', 'mejor_val',
                       'numindiv', 'maxiter', 'early', 'Lambda', 'c1', 'c2', 'IWmax', 'IWmin',
                       'K', 'globthr', 'CVrep', 'CVspl', 'nruns', 'pcrossover_elitists',
                       'pcrossover', 'best_solution']].head(1),
                 pd.DataFrame(pd.concat([fil0,fil1])).transpose()],axis=1)

            # Model with mean of hyperparameters
            # Entrenamos con TRAIN+VAL y medimos el error con TST
            model = res_search_model.best_model
            model.hidden_layer_sizes = int(mean_outer['mean_hidden'].values[0])
            model.alpha = mean_outer['mean_alpha'].values[0]

        if arg_in.algorithm == 'kridge':
            only_outer = res_final.query('num_seed==@num_seed').reset_index(drop=True)
            only_outer['complex_best'] =  only_outer['best_solution'].apply(lambda x: x[2]).values
            only_outer['alpha'] =  only_outer['best_solution'].apply(lambda x: 10**x[3]).values
            only_outer['gamma'] =  only_outer['best_solution'].apply(lambda x: 10**x[4]).values
            mean_outer = only_outer[['VAL_RMSE_FOLD', 'VAL_MAE', 'VAL_MAPE', 'TST_RMSE_FOLD',
                                     'TST_RMSE', 'TST_MAE', 'TST_MAPE',
                                    'complex_best', 'alpha', 'gamma']].aggregate(['mean','std'])
            fil0 = mean_outer.iloc[0]
            fil0.index = ['mean_'+col for col in mean_outer.columns]
            fil1 = mean_outer.iloc[1]
            fil1.index = ['std_'+col for col in mean_outer.columns]
            mean_outer = pd.concat(
                [only_outer[['method', 'database', 'namevar', 'size_train_val', 'size_test',
                       'num_input_feats', 'num_seed', 'innerfold', 'run', 'model_name',
                       'rerank_error', 'time', 'last_iter', 'mejor_val',
                       'numindiv', 'maxiter', 'early', 'Lambda', 'c1', 'c2', 'IWmax', 'IWmin',
                       'K', 'globthr', 'CVrep', 'CVspl', 'nruns', 'pcrossover_elitists',
                       'pcrossover', 'best_solution']].head(1),
                 pd.DataFrame(pd.concat([fil0,fil1])).transpose()],axis=1)

            # Model with mean of hyperparameters
            # Entrenamos con TRAIN+VAL y medimos el error con TST
            model = res_search_model.best_model
            model.alpha = mean_outer['mean_alpha'].values[0]
            model.gamma = mean_outer['mean_gamma'].values[0]

        model.fit(X_train_val[selec_input].values, y_train_val.values)
        y_test_pred = model.predict(X_test[selec_input].values)
        OUTER_RMSE = mean_squared_error(y_test.values, y_test_pred, squared=False)
        OUTER_MAE = mean_absolute_error(y_test.values, y_test_pred)
        OUTER_MAPE = mean_absolute_percentage_error(y_test.values, y_test_pred) * 100
        mean_outer['OUTER_RMSE'] = OUTER_RMSE
        mean_outer['OUTER_MAE'] = OUTER_MAE
        mean_outer['OUTER_MAPE'] = OUTER_MAPE
        outer_res.append(mean_outer)
    outer_res = pd.concat(outer_res)
    outer_res.to_csv(DIR_SALIDA + '/' + NAME_FILE + '_' + name_db.split('_')[0] + '_outer.csv', index=False)
    print(name_db)
    display(outer_res[['mean_VAL_RMSE_FOLD', 'mean_TST_RMSE_FOLD', 'mean_TST_RMSE', 'OUTER_RMSE']].mean())
    print('##########################################################################')

## Obtain Outer Models by Selecting Features

In [ ]:
path_files = 'hyb_1001_12abr23_e27/'
archivos = os.listdir(path_files)
archivos

### Bayesian with best features

In [ ]:
# Optimize with BayesianOptimization
def bayes_cv(chromosome_all_ones):
    global X_train_val, y_train_val
    res_folds = []
    kf = KFold(n_splits=arg_in.numseeds, shuffle=True, random_state=1234)
    for i, (train_index, val_index) in enumerate(kf.split(X_train_val)):
        X_train = X_train_val.iloc[train_index].reset_index(drop=True)
        y_train = y_train_val.iloc[train_index].reset_index(drop=True)
        X_val = X_train_val.iloc[val_index].reset_index(drop=True)
        y_val = y_train_val.iloc[val_index].reset_index(drop=True)
        res_folds.append(fitness_fun(chromosome_all_ones, X_train, y_train, X_val, y_val, 0, 0)[0][1])
    res_folds = np.array(res_folds)
    return np.mean(res_folds)

In [ ]:
def optimize_fun(hidden_layer_sizes, alpha):
    params_opt = dict(hidden_layer_sizes=int(hidden_layer_sizes),
                      alpha=alpha,
                      solver='lbfgs',
                      activation='logistic',
                      n_iter_no_change=20,
                      tol=1e-5,
                      random_state=1234,
                      max_iter=5000)
    chromosome_all_ones = pd.Series(dict(columns=(np.ones(len(input_names)) == 1), params=params_opt))
    return bayes_cv(chromosome_all_ones)

In [ ]:
# Ridge
if arg_in.algorithm == 'ridge':
    def optimize_fun(alpha):
        chromosome_all_ones = pd.Series(
            dict(columns=(np.ones(len(input_names)) == 1), params=dict(alpha=alpha, tol=1e-4)))
        return bayes_cv(chromosome_all_ones)  
    
# KernelRidge
if arg_in.algorithm == 'kridge':
    def optimize_fun(alpha, gamma):
        chromosome_all_ones = pd.Series(
            dict(columns=(np.ones(len(input_names)) == 1), params=dict(alpha=alpha, gamma=gamma, kernel="rbf")))
        return bayes_cv(chromosome_all_ones)  # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.

# SVR
if arg_in.algorithm == 'svr':
    def optimize_fun(C, gamma):
        chromosome_all_ones = pd.Series(
            dict(columns=(np.ones(len(input_names)) == 1), params=dict(C=C, gamma=gamma, kernel="rbf")))
        return bayes_cv(chromosome_all_ones)  # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.
    
# DecisionTreeRegressor
if arg_in.algorithm == 'dtr':
    def optimize_fun(max_depth, min_impurity_decrease):
        chromosome_all_ones = pd.Series(
            dict(columns=(np.ones(len(input_names)) == 1), params=dict(max_depth=max_depth, min_impurity_decrease=min_impurity_decrease)))
        return bayes_cv(chromosome_all_ones)  # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.   
    
# RandomForest
if arg_in.algorithm == 'rf':
    def optimize_fun(max_depth, n_estimators, min_impurity_decrease):
        chromosome_all_ones = pd.Series(
            dict(columns=(np.ones(len(input_names)) == 1), params=dict(max_depth=max_depth, 
                                                                       n_estimators=n_estimators,
                                                                       min_impurity_decrease=min_impurity_decrease)))
        return bayes_cv(chromosome_all_ones)  # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.        

# MultilayerPerceptron
if arg_in.algorithm == 'mlp':
    def optimize_fun(hidden_layer_sizes, alpha):
        params_opt = dict(hidden_layer_sizes=int(hidden_layer_sizes),
                          alpha=alpha,
                          solver='lbfgs',
                          activation='logistic',
                          n_iter_no_change=20,
                          tol=1e-5,
                          random_state=1234,
                          max_iter=5000)
        chromosome_all_ones = pd.Series(dict(columns=(np.ones(len(input_names)) == 1), params=params_opt))
        return bayes_cv(chromosome_all_ones) # Antes era res[0][0], pero ahora nos quedamos con res[0][1] porque queremos el error en X_val.

def desnorm_y(y_pred):
    return ((y_pred*y_orig_sqrt_std)+y_orig_sqrt_mean)**2.0

In [ ]:
for thr_features in [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70]:
    run = 0
    features_list = []
    for names_tiempo in tiempos:
        name_db = names_tiempo[0].split('_')[0]
        list_csv = os.listdir('.')
        list_csv = [i for i in list_csv if f'{name_db}' in i]
        file_db = list_csv[0]
        time_limit = int(names_tiempo[1] * 60)  # Time limit in secs
        train_val_size = names_tiempo[2]
        print(name_db, time_limit, train_val_size)

        NAME_FILE = dt_string + '_' + arg_in.method + '__' + arg_in.algorithm + '__' + file_db + '__' + str(arg_in.nump)  # .join(arg_in[:2]) #+ str_todos
        NAME_FILE = ''.join(char for char in NAME_FILE if char in printable)
        print(repr(NAME_FILE))

        res_modelos = []
        GA_models = []
        historical_todos = []
        tiempo_inicial = time.time()
        outer_res = []

        for num_seed in range(arg_in.numseeds):
            print('Procesando Base de Datos con num_seed=', num_seed)
            df_VAL = pd.read_csv(file_db)
            print(df_VAL.shape)
            train_val_size = df_VAL.shape[0]//2
            if train_val_size>2000:
                train_val_size = 2000
            names_tiempo[2] = train_val_size
            print(name_db, time_limit, train_val_size)
            num_feats = df_VAL.shape[1]-1

            df_VAL = pd.read_csv(file_db)
            VAL = df_VAL.copy()
            scaler_pruebas = StandardScaler()
            VAL_norm = pd.DataFrame(scaler_pruebas.fit_transform(VAL), columns=VAL.columns)

            TST_RMSE_ITER = []
            NUM_FEATS = []

            # Con selección
            input_names = VAL_norm.columns[:-1]
            target_name = VAL_norm.columns[-1]

            X = VAL_norm[input_names]
            y = VAL_norm[target_name]
            seed_everything(num_seed)

             # IMPORTANTE: En KFOLD_HYB_AUTOGLUON cojo 2000 filas de train_val y el resto de test...salvo que se especifique
            # el parámetro train_size (que será el número de filas a pillar de la BD).
            X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, 
                                                                        train_size = names_tiempo[2] / len(X), 
                                                                        random_state=12345)
            size_train_val = len(X_train_val)
            size_test = len(X_test)

            model_name = 'KFOLD_HYB_AUTOGLUON'
            selec_input_folds=[]
            VAL_RMSE_ACC=[]
            # En este caso, hacemos los X folds (fold es el numseeds)
            kf = KFold(n_splits=arg_in.numseeds, shuffle=False)

            file_name = [i for i in archivos if name_db in i and 'results' in i][0]
            results_name = path_files + file_name
            res_final = pd.read_csv(results_name) #DIR_SALIDA + '/' + NAME_FILE + '_results.csv')
    #         display(res_final)

            def convierte_str(cadena):
                return np.array(eval(cadena.replace(' ',',').replace('\n','').replace('nan','0')))


            # FINAL MODEL FOR EACH OUTER WITH MEAN PARAMS
            if arg_in.algorithm == 'mlp':
                only_outer = res_final.query('num_seed==@num_seed').reset_index(drop=True)
                only_outer['best_solution'] = only_outer['best_solution'].apply(lambda x: convierte_str(x))
                only_outer['complex_best'] =  only_outer['best_solution'].apply(lambda x: x[2]).values
                only_outer['hidden'] =  only_outer['best_solution'].apply(lambda x: x[3]).values
                only_outer['alpha'] =  only_outer['best_solution'].apply(lambda x: 10**x[4]).values
                only_outer['features'] =  only_outer['best_solution'].apply(lambda x: x[-len(input_names):]).values
                mean_outer = only_outer[['VAL_RMSE_FOLD', 'VAL_MAE', 'VAL_MAPE', 'TST_RMSE_FOLD',
                                         'TST_RMSE', 'TST_MAE', 'TST_MAPE',
                                        'complex_best', 'hidden', 'alpha']].aggregate(['mean','std'])
                fil0 = mean_outer.iloc[0]
                fil0.index = ['mean_'+col for col in mean_outer.columns]
                fil1 = mean_outer.iloc[1]
                fil1.index = ['std_'+col for col in mean_outer.columns]
                mean_outer = pd.concat(
                    [only_outer[['method', 'database', 'namevar', 'size_train_val', 'size_test',
                           'num_input_feats', 'num_seed', 'innerfold', 'run', 'model_name',
                           'rerank_error', 'time', 'last_iter', 'mejor_val',
                           'numindiv', 'maxiter', 'early', 'Lambda', 'c1', 'c2', 'IWmax', 'IWmin',
                           'K', 'globthr', 'CVrep', 'CVspl', 'nruns', 'pcrossover_elitists',
                           'pcrossover', 'best_solution']].head(1),
                     pd.DataFrame(pd.concat([fil0,fil1])).transpose()],axis=1)

                # Model with mean of hyperparameters
                # Entrenamos con TRAIN+VAL y medimos el error con TST
                model = MLPRegressor()
                model.hidden_layer_sizes = int(mean_outer['mean_hidden'].values[0])
                model.alpha = mean_outer['mean_alpha'].values[0]

            if arg_in.algorithm == 'kridge':
                only_outer = res_final.query('num_seed==@num_seed').reset_index(drop=True)
                only_outer['best_solution'] = only_outer['best_solution'].apply(lambda x: convierte_str(x))
                only_outer['complex_best'] =  only_outer['best_solution'].apply(lambda x: x[2]).values
                only_outer['alpha'] =  only_outer['best_solution'].apply(lambda x: 10**x[3]).values
                only_outer['gamma'] =  only_outer['best_solution'].apply(lambda x: 10**x[4]).values
                only_outer['features'] =  only_outer['best_solution'].apply(lambda x: x[-len(input_names):]).values
                mean_outer = only_outer[['VAL_RMSE_FOLD', 'VAL_MAE', 'VAL_MAPE', 'TST_RMSE_FOLD',
                                         'TST_RMSE', 'TST_MAE', 'TST_MAPE',
                                        'complex_best', 'alpha', 'gamma']].aggregate(['mean','std'])
                fil0 = mean_outer.iloc[0]
                fil0.index = ['mean_'+col for col in mean_outer.columns]
                fil1 = mean_outer.iloc[1]
                fil1.index = ['std_'+col for col in mean_outer.columns]
                mean_outer = pd.concat(
                    [only_outer[['method', 'database', 'namevar', 'size_train_val', 'size_test',
                           'num_input_feats', 'num_seed', 'innerfold', 'run', 'model_name',
                           'rerank_error', 'time', 'last_iter', 'mejor_val',
                           'numindiv', 'maxiter', 'early', 'Lambda', 'c1', 'c2', 'IWmax', 'IWmin',
                           'K', 'globthr', 'CVrep', 'CVspl', 'nruns', 'pcrossover_elitists',
                           'pcrossover', 'best_solution']].head(1),
                     pd.DataFrame(pd.concat([fil0,fil1])).transpose()],axis=1)

                # Model with mean of hyperparameters
                # Entrenamos con TRAIN+VAL y medimos el error con TST
                model = res_search_model.best_model
                model.alpha = mean_outer['mean_alpha'].values[0]
                model.gamma = mean_outer['mean_gamma'].values[0]

    #         selec_features = input_names[(np.stack(only_outer['features'].values)>0.50).mean(axis=0)>=0.50]
            selec_features = input_names[(np.mean(np.stack(only_outer['features'].values),axis=0)>= thr_features)]
            print(f'DB={name_db} train_val_size={train_val_size} ncols={len(input_names)} NFS={len(selec_features)}')
            features_list.append(dict(name_db=name_db,
                                      num_feats=num_feats,
                                      num_selec = len(selec_features),

                                      file_db=file_db,
                                      results_name=results_name,
                                      selec_features=np.array(selec_features)
                                     ))

    features_list = pd.DataFrame(features_list)

    # Search with Bayesian
    arg_in.maxiter = 200
    run = 0
    res = []
    for names_tiempo in tiempos:
        name_db = names_tiempo[0].split('_')[0]
        list_csv = os.listdir('.')
        list_csv = [i for i in list_csv if f'{name_db}' in i]
        file_db = list_csv[0]
        time_limit = int(names_tiempo[1] * 60)  # Time limit in secs
        train_val_size = names_tiempo[2]
        print(name_db, time_limit, train_val_size)

        NAME_FILE = dt_string + '_' + arg_in.method + '__' + arg_in.algorithm + '__' + file_db + '__' + str(arg_in.nump)  # .join(arg_in[:2]) #+ str_todos
        NAME_FILE = ''.join(char for char in NAME_FILE if char in printable)
        print(repr(NAME_FILE))

        res_modelos = []
        GA_models = []
        historical_todos = []
        tiempo_inicial = time.time()
        outer_res = []

        for num_seed in range(arg_in.numseeds):
            print('Procesando Base de Datos=', name_db,' con num_seed=', num_seed, 'thr_features=', thr_features)
            df_VAL = pd.read_csv(file_db)
            print(df_VAL.shape)
            train_val_size = df_VAL.shape[0]//2
            if train_val_size>2000:
                train_val_size = 2000
            names_tiempo[2] = train_val_size

            num_feats = df_VAL.shape[1]-1

            df_VAL = pd.read_csv(file_db)
            VAL = df_VAL.copy()
            scaler_pruebas = StandardScaler()
            VAL_norm = pd.DataFrame(scaler_pruebas.fit_transform(VAL), columns=VAL.columns)

            TST_RMSE_ITER = []
            NUM_FEATS = []

            # Con selección
    #         input_names = VAL_norm.columns[:-1]
            input_names = features_list.query('name_db==@name_db').iloc[num_seed]['selec_features']
            target_name = VAL_norm.columns[-1]
            print(f'DB={name_db} train_val_size={train_val_size} ncols={df_VAL.shape[1]-1} NFS={len(input_names)}')
            if len(input_names)==0:
                input_names = VAL_norm.columns[:2]
            X = VAL_norm[input_names]
            y = VAL_norm[target_name]
            seed_everything(num_seed)

            # IMPORTANTE: En KFOLD_HYB_AUTOGLUON cojo 2000 filas de train_val y el resto de test...salvo que se especifique
            # el parámetro train_size (que será el número de filas a pillar de la BD).
            X_train_val, X_test, y_train_val, y_test = train_test_split(X[input_names], y, 
                                                                        train_size = names_tiempo[2] / len(X), 
                                                                        random_state=12345)
            size_train_val = len(X_train_val)
            size_test = len(X_test)

            model_name = 'KFOLD_HYB_AUTOGLUON'
            selec_input_folds=[]
            VAL_RMSE_ACC=[]

            # Search With CV
            # --------------
            pbounds = dict()
            for parm in params:
                if params[parm]['type'] <= 1:
                    pbounds[parm] = params[parm]['range']

            tic = time.time()
            optimizer = BayesianOptimization(f=optimize_fun,
                                             pbounds=pbounds,
                                             random_state=1234,
                                             allow_duplicate_points=True) # Añado allow_duplicate_points=True para evitar errores que salían.
            optimizer.maximize(init_points=10, n_iter=arg_in.maxiter - 10)
            # Keep elapsed time in minutes
            tac = time.time()
            elapsed_time = (tac - tic) / 60.0
            print('Elapsed_time:', elapsed_time)
            print('Best:', optimizer.max)

            if arg_in.algorithm == 'mlp':
                # Best model
                hidden_layer_sizes = optimizer.max['params']['hidden_layer_sizes']
                alpha = 10.0**optimizer.max['params']['alpha']
                params_opt = dict(hidden_layer_sizes=int(hidden_layer_sizes),
                                  alpha=alpha,
                                  solver='lbfgs',
                                  activation='logistic',
                                  n_iter_no_change=20,
                                  tol=1e-5,
                                  random_state=1234,
                                  max_iter=5000)
                params_indiv = [optimizer.max['params']['hidden_layer_sizes'], optimizer.max['params']['alpha']]
                model = MLPRegressor(**params_opt)
            
                # Entrena con innerfolds y saca el error con test
                model.fit(X_train_val[input_names].values, y_train_val.values)
                y_test_pred = model.predict(X_test[input_names].values)
                OUTER_RMSE = mean_squared_error(y_test.values, y_test_pred, squared=False)
                OUTER_MAE = mean_absolute_error(y_test.values, y_test_pred)
                OUTER_MAPE = mean_absolute_percentage_error(y_test.values, y_test_pred) * 100
                print(OUTER_RMSE, OUTER_MAE, OUTER_MAPE)
                res.append(dict(name_db=name_db,
                                CV_RMSE = -optimizer.max['target'],
                                OUTER_RMSE=OUTER_RMSE,
                                OUTER_MAE=OUTER_MAE,
                                OUTER_MAPE=OUTER_MAPE,
                                train_val_size=train_val_size,
                                test_size=len(X_test),
                                ncols=df_VAL.shape[1]-1,
                                NFs=len(input_names),
                                hidden_layer_sizes=hidden_layer_sizes,
                                alpha=alpha,
                                num_seed=num_seed,
                                input_names=input_names))

            if arg_in.algorithm == 'kridge':
                # Best model
                alpha = 10.0**optimizer.max['params']['alpha']
                gamma = 10.0**optimizer.max['params']['gamma']
                params_opt = dict(alpha=alpha, 
                                  gamma=gamma, 
                                  kernel="rbf")
                params_indiv = [optimizer.max['params']['alpha'], optimizer.max['params']['gamma']]
                model = KernelRidge(**params_opt)
                
                # Entrena con innerfolds y saca el error con test
                model.fit(X_train_val[input_names].values, y_train_val.values)
                y_test_pred = model.predict(X_test[input_names].values)
                OUTER_RMSE = mean_squared_error(y_test.values, y_test_pred, squared=False)
                OUTER_MAE = mean_absolute_error(y_test.values, y_test_pred)
                OUTER_MAPE = mean_absolute_percentage_error(y_test.values, y_test_pred) * 100
                print(OUTER_RMSE, OUTER_MAE, OUTER_MAPE)
                res.append(dict(name_db=name_db,
                                CV_RMSE = -optimizer.max['target'],
                                OUTER_RMSE=OUTER_RMSE,
                                OUTER_MAE=OUTER_MAE,
                                OUTER_MAPE=OUTER_MAPE,
                                train_val_size=train_val_size,
                                test_size=len(X_test),
                                ncols=df_VAL.shape[1]-1,
                                NFs=len(input_names),
                                alpha=alpha,
                                gamma=gamma,
                                num_seed=num_seed,
                                input_names=input_names))
            res_total = pd.DataFrame(res)
            res_total.to_csv(f'res_bayes_after_pso_thrfeats_{thr_features}.csv', index=False)

In [ ]:
res_total.groupby('name_db').mean()

### Analysis

In [ ]:
features_list_thr = []
for thr_features in [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70]:
    run = 0
    for names_tiempo in tiempos:
        name_db = names_tiempo[0].split('_')[0]
        list_csv = os.listdir('.')
        list_csv = [i for i in list_csv if f'{name_db}' in i]
        file_db = list_csv[0]
        time_limit = int(names_tiempo[1] * 60)  # Time limit in secs
        train_val_size = names_tiempo[2]
        print(name_db, time_limit, train_val_size)

        NAME_FILE = dt_string + '_' + arg_in.method + '__' + arg_in.algorithm + '__' + file_db + '__' + str(arg_in.nump)  # .join(arg_in[:2]) #+ str_todos
        NAME_FILE = ''.join(char for char in NAME_FILE if char in printable)
        print(repr(NAME_FILE))

        res_modelos = []
        GA_models = []
        historical_todos = []
        tiempo_inicial = time.time()
        outer_res = []

        for num_seed in range(arg_in.numseeds):
            print('Procesando Base de Datos con num_seed=', num_seed)
            df_VAL = pd.read_csv(file_db)
            print(df_VAL.shape)
            train_val_size = df_VAL.shape[0]//2
            if train_val_size>2000:
                train_val_size = 2000
            names_tiempo[2] = train_val_size
            print(name_db, time_limit, train_val_size)
            num_feats = df_VAL.shape[1]-1

            df_VAL = pd.read_csv(file_db)
            VAL = df_VAL.copy()
            scaler_pruebas = StandardScaler()
            VAL_norm = pd.DataFrame(scaler_pruebas.fit_transform(VAL), columns=VAL.columns)

            TST_RMSE_ITER = []
            NUM_FEATS = []

            # Con selección
            input_names = VAL_norm.columns[:-1]
            target_name = VAL_norm.columns[-1]

            X = VAL_norm[input_names]
            y = VAL_norm[target_name]
            seed_everything(num_seed)

             # IMPORTANTE: En KFOLD_HYB_AUTOGLUON cojo 2000 filas de train_val y el resto de test...salvo que se especifique
            # el parámetro train_size (que será el número de filas a pillar de la BD).
            X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, 
                                                                        train_size = names_tiempo[2] / len(X), 
                                                                        random_state=12345)
            size_train_val = len(X_train_val)
            size_test = len(X_test)

            model_name = 'KFOLD_HYB_AUTOGLUON'
            selec_input_folds=[]
            VAL_RMSE_ACC=[]
            # En este caso, hacemos los X folds (fold es el numseeds)
            kf = KFold(n_splits=arg_in.numseeds, shuffle=False)

            file_name = [i for i in archivos if name_db in i and 'results' in i][0]
            results_name = path_files + file_name
            res_final = pd.read_csv(results_name) #DIR_SALIDA + '/' + NAME_FILE + '_results.csv')
    #         display(res_final)

            def convierte_str(cadena):
                return np.array(eval(cadena.replace(' ',',').replace('\n','').replace('nan','0')))


            # FINAL MODEL FOR EACH OUTER WITH MEAN PARAMS
            if arg_in.algorithm == 'mlp':
                only_outer = res_final.query('num_seed==@num_seed').reset_index(drop=True)
                only_outer['best_solution'] = only_outer['best_solution'].apply(lambda x: convierte_str(x))
                only_outer['complex_best'] =  only_outer['best_solution'].apply(lambda x: x[2]).values
                only_outer['hidden'] =  only_outer['best_solution'].apply(lambda x: x[3]).values
                only_outer['alpha'] =  only_outer['best_solution'].apply(lambda x: 10**x[4]).values
                only_outer['features'] =  only_outer['best_solution'].apply(lambda x: x[-len(input_names):]).values
                mean_outer = only_outer[['VAL_RMSE_FOLD', 'VAL_MAE', 'VAL_MAPE', 'TST_RMSE_FOLD',
                                         'TST_RMSE', 'TST_MAE', 'TST_MAPE',
                                        'complex_best', 'hidden', 'alpha']].aggregate(['mean','std'])
                fil0 = mean_outer.iloc[0]
                fil0.index = ['mean_'+col for col in mean_outer.columns]
                fil1 = mean_outer.iloc[1]
                fil1.index = ['std_'+col for col in mean_outer.columns]
                mean_outer = pd.concat(
                    [only_outer[['method', 'database', 'namevar', 'size_train_val', 'size_test',
                           'num_input_feats', 'num_seed', 'innerfold', 'run', 'model_name',
                           'rerank_error', 'time', 'last_iter', 'mejor_val',
                           'numindiv', 'maxiter', 'early', 'Lambda', 'c1', 'c2', 'IWmax', 'IWmin',
                           'K', 'globthr', 'CVrep', 'CVspl', 'nruns', 'pcrossover_elitists',
                           'pcrossover', 'best_solution']].head(1),
                     pd.DataFrame(pd.concat([fil0,fil1])).transpose()],axis=1)

                # Model with mean of hyperparameters
                # Entrenamos con TRAIN+VAL y medimos el error con TST
                model = MLPRegressor()
                model.hidden_layer_sizes = int(mean_outer['mean_hidden'].values[0])
                model.alpha = mean_outer['mean_alpha'].values[0]

            if arg_in.algorithm == 'kridge':
                only_outer = res_final.query('num_seed==@num_seed').reset_index(drop=True)
                only_outer['best_solution'] = only_outer['best_solution'].apply(lambda x: convierte_str(x))
                only_outer['complex_best'] =  only_outer['best_solution'].apply(lambda x: x[2]).values
                only_outer['alpha'] =  only_outer['best_solution'].apply(lambda x: 10**x[3]).values
                only_outer['gamma'] =  only_outer['best_solution'].apply(lambda x: 10**x[4]).values
                only_outer['features'] =  only_outer['best_solution'].apply(lambda x: x[-len(input_names):]).values
                mean_outer = only_outer[['VAL_RMSE_FOLD', 'VAL_MAE', 'VAL_MAPE', 'TST_RMSE_FOLD',
                                         'TST_RMSE', 'TST_MAE', 'TST_MAPE',
                                        'complex_best', 'alpha', 'gamma']].aggregate(['mean','std'])
                fil0 = mean_outer.iloc[0]
                fil0.index = ['mean_'+col for col in mean_outer.columns]
                fil1 = mean_outer.iloc[1]
                fil1.index = ['std_'+col for col in mean_outer.columns]
                mean_outer = pd.concat(
                    [only_outer[['method', 'database', 'namevar', 'size_train_val', 'size_test',
                           'num_input_feats', 'num_seed', 'innerfold', 'run', 'model_name',
                           'rerank_error', 'time', 'last_iter', 'mejor_val',
                           'numindiv', 'maxiter', 'early', 'Lambda', 'c1', 'c2', 'IWmax', 'IWmin',
                           'K', 'globthr', 'CVrep', 'CVspl', 'nruns', 'pcrossover_elitists',
                           'pcrossover', 'best_solution']].head(1),
                     pd.DataFrame(pd.concat([fil0,fil1])).transpose()],axis=1)

                # Model with mean of hyperparameters
                # Entrenamos con TRAIN+VAL y medimos el error con TST
                model = res_search_model.best_model
                model.alpha = mean_outer['mean_alpha'].values[0]
                model.gamma = mean_outer['mean_gamma'].values[0]

    #         selec_features = input_names[(np.stack(only_outer['features'].values)>0.50).mean(axis=0)>=0.50]
            selec_features = input_names[(np.mean(np.stack(only_outer['features'].values),axis=0)>= thr_features)]
            print(f'DB={name_db} train_val_size={train_val_size} ncols={len(input_names)} NFS={len(selec_features)}')
            features_list_thr.append(dict(name_db=name_db,
                                          num_seed = num_seed,
                                          thr_features=thr_features,
                                          num_feats=num_feats,
                                          mean_last_iter = mean_outer['last_iter'].values[0],
                                          num_selec = len(selec_features),

                                          file_db=file_db,
                                          results_name=results_name,
                                          selec_features=np.array(selec_features)
                                     ))

features_list_thr = pd.DataFrame(features_list_thr)
features_list_thr.groupby(['name_db','thr_features']).mean()

In [ ]:
features_list_thr_mean = features_list_thr.groupby(['name_db','thr_features']).mean().reset_index()
features_list_thr_mean

In [ ]:
features_list_thr_mean.query('thr_features==@thr_features')

In [ ]:
for thr_features in [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70]:
    res_total = pd.read_csv(f'res_bayes_after_pso_thrfeats_{thr_features}.csv')
    mean_res = res_total.groupby('name_db').mean().reset_index()
    if thr_features == 0.00:
        db_mean = mean_res[['name_db', 'train_val_size', 'test_size', 'ncols']]
        db_mean.rename({'OUTER_RMSE':'RMSE_0.0'}, inplace=True)
        db_mean['last_iter'] = features_list_thr_mean.query('thr_features==@thr_features')['mean_last_iter'].values
    db_mean[f'RMSE_{thr_features}'] = mean_res['OUTER_RMSE']
    db_mean[f'NFs_{thr_features}'] = features_list_thr_mean.query('thr_features==@thr_features')['num_selec'].values
db_mean                   
    

In [ ]:
print(db_mean.to_latex())

In [ ]:
features_list_thr_std = features_list_thr.groupby(['name_db','thr_features']).std().reset_index()
features_list_thr_std

In [ ]:
for thr_features in [0.00, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70]:
    res_total = pd.read_csv(f'res_bayes_after_pso_thrfeats_{thr_features}.csv')
    std_res = res_total.groupby('name_db').std().reset_index()
    if thr_features == 0.00:
        db_std = std_res[['name_db', 'train_val_size', 'test_size', 'ncols']]
        db_std.rename({'OUTER_RMSE':'RMSE_0.0'}, inplace=True)
        db_std['last_iter'] = features_list_thr_std.query('thr_features==@thr_features')['mean_last_iter'].values
    db_std[f'RMSE_{thr_features}'] = std_res['OUTER_RMSE']
    db_std[f'NFs_{thr_features}'] = features_list_thr_std.query('thr_features==@thr_features')['num_selec'].values
db_std                   
    

In [ ]:
db_std[['name_db', 'RMSE_0.5', 'NFs_0.5']]